In [281]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep
import json
from pymorphy2 import MorphAnalyzer
import re
from collections import OrderedDict
from dictdiffer import diff, patch, swap, revert

from selenium.webdriver import Firefox, FirefoxProfile, Proxy
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.proxy import ProxyType

from lxml import html

In [282]:
class SiteParser:
    def __init__(self):
        pass
    def get_morphems(self, word):
        pass
    def __del__(self):
        pass
    
class SiteParser1(SiteParser):
    def get_morphems(self, word):
        """Получает морфемы слова
        Обращается по адресу и выдает морфемы в виде списка [(морфема1, название1),(морфема2, название2), ...]
        """
        url = "https://kartaslov.ru/разбор-слова-по-составу/{}".format(word)
        text = requests.get(url).text
        soup = BeautifulSoup(text, 'html.parser')
        
        # у сайта проблемы, например, со словом пять, парсит другое слово(пятить), поэтому чекаем на совпадение с желаемым
        # сейчас просто ингорим, но может стоит выкидывать то что лишнее добавляется - надо исследовать
        word_on_site = re.match(".*«(.*)»", soup.find('h1', class_='v2-h1').text)
        if word_on_site.group(1) != word:
            return {"error":"site parsing error"}

        table = soup.find('table', class_='morphemics-table')
        if table is None:
            return {"error":"not found on site"}
        
        rows = table.findAll('tr')
        info = []
        for row in rows:
            info.append((row.find('td', class_='td-morpheme-text').text, row.find('td', class_='td-morpheme-type').text))
         
        return info  
    
class SiteParser2(SiteParser):
    def __init__(self):
        opts = Options()
        opts.set_headless()
        assert opts.headless
        #self.browser = Firefox(options=opts)
        
        my_proxy = Proxy({'proxyType': ProxyType.MANUAL,
        'httpProxy': '123.49.51.90:40881',
        'sslProxy': '123.49.51.90:40881',
        'noProxy': 'www.google-analytics.com, ajax.googleapis.com, apis.google.com'
        })

        self.browser = Firefox(proxy=my_proxy, options=opts, executable_path='C:\\Users\\NAD-ErmolaevPA\\Documents\\kitty_kursach\\geckodriver.exe')
        self.browser.get('https://udarenieru.ru/index.php?word=on')
        
    def get_morphems(self, word):
        """Получает морфемы слова
        Обращается по адресу и выдает морфемы в виде списка [(морфема1, название1),(морфема2, название2), ...]
        """
        self.browser.get('https://udarenieru.ru/index.php?word=on')
        self.browser.find_element_by_id('word_db').send_keys(word)
        self.browser.find_element_by_id('morph_book_D24').click()

        results = self.browser.find_elements_by_xpath('//*[@id="kuz_interpret"]/tbody/tr[1]/td/table')
        if results == []:
            return {"error":"not found on site"}
        res = results[0].text

        return [[x.strip() for x in y.split('—')] for y in res.split('\n')]
    
    def __del__(self):
        self.browser.close()
        
        
class SiteParser3(SiteParser):
    def get_morphems(self, word):
        """Получает морфемы слова
        Обращается по адресу и выдает морфемы в виде списка [(морфема1, название1),(морфема2, название2), ...]
        """
        url = "http://sostavslova.ru/{}/{}".format(word[0].upper(), word)
        text = requests.get(url).text
        
        soup = BeautifulSoup(text, 'html.parser')
        table = soup.find('p')
        if table is None:
            return {"error":"not found on site"}
        
        info = []
        mapper = {'w-pref':'приставка', 'w-root':'корень', 'w-suff':'суффикс', 'w-end':'окончание', 'w-end-null':'нулевое окончание', 'w-post':'постфикс'}
        for row in table.findAll('span'):
            cl = row.attrs['class'][0]
            text = row.text
            if cl.find('w-')!=-1:
                info.append((text, mapper[cl]))
                
        if info==[]:
            return {"error":"not found on site"}
        return info   

In [168]:
url = "http://sostavslova.ru/{}/{}".format(word[0].upper(), word)
text = requests.get(url).text
soup = BeautifulSoup(text, 'html.parser')
table = soup.find('p')
info = []
mapper = {'w-pref':'приставка', 'w-root':'корень', 'w-suff':'суффикс', 'w-end':'окончание', 'w-end-null':'нулевое окончание'}
for row in table.findAll('span'):
    cl = row.attrs['class'][0]
    text = row.text
    if cl.find('w-')!=-1:
        info.append((mapper[cl], text))

In [169]:
info

[('приставка', 'при'),
 ('корень', 'чес'),
 ('суффикс', 'а'),
 ('суффикс', 'ть'),
 ('суффикс', 'ся')]

In [162]:
table.findAll('span')

[<span class="word"><span class="base"><span class="w-pref">при</span><span class="w-root">чес</span><span class="w-suff">а</span></span><span class="w-suff">ть</span><span class="base"><span class="w-suff">ся</span></span></span>,
 <span class="base"><span class="w-pref">при</span><span class="w-root">чес</span><span class="w-suff">а</span></span>,
 <span class="w-pref">при</span>,
 <span class="w-root">чес</span>,
 <span class="w-suff">а</span>,
 <span class="w-suff">ть</span>,
 <span class="base"><span class="w-suff">ся</span></span>,
 <span class="w-suff">ся</span>]

In [448]:
class Analyzer:
    def __init__(self, site_parsers):
        self._analyzer = MorphAnalyzer()
        self._site_parsers = site_parsers

    def get_POS(self, word, analyzer='pymorphy'):
        """Получает часть речи для слова
        Для многозначных берем первое что показывает анализатор, возмножны проблемы
        Заложен вариант для остальных анализаторов, пока не реализован
        """
        if analyzer=='pymorphy':
            r = self._analyzer.parse(word)[0].tag.POS
            return r if r!='COMP' else 'ADJF'
        else:
            raise NotImplementedError
    
    def _get_dd(self, m):
        """Переводит список морфем и их названий в упорядоченный словарь
        """
        d = OrderedDict()
        for y in m:
            if len(y)==2:
                v, k = y
                v = v.replace('ё', 'е')
                if k in d:
                    d[k].append(v)
                else:
                    d[k] = [v]
        return d.copy()
    
    def _find(self, dif, action='add', morpheme='приставка'):
        """Ищет в dictdiffer различия определенного типа
        """
        for v in dif:
            if v[0]==action:
                if v[1]==morpheme: 
                    return True
                elif type(v[1])==list:
                    if v[1][0]==morpheme: 
                        return v[2]                  
                elif v[1]=='':
                    # thats in case [('add', '', [('приставка', ['при']), ('суффикс', ['ва'])])]
                    for i in v[2]:
                        if i[0]==morpheme:
                            return True
        return False

    def analyze(self, word1, word2, verbose=True):
        """Анализ слов на пердмет слообразования
        """
        #get morphemes
        
        for site_parser in self._site_parsers:
            m1 = site_parser.get_morphems(word1)
            if type(m1)!=dict:
                break
        
        for site_parser in self._site_parsers:
            m2 = site_parser.get_morphems(word2)
            if type(m2)!=dict:
                break
                
        
        #check if morphemes valid
        if 'error' in m1:
            if verbose: print('cant parse {}, error:'.format(word1, m1['error']))
            return 'UNKNOWN_PARSE_ERROR'       
        if 'error' in m2:
            if verbose: print('cant parse {}, error:'.format(word2, m2['error']))
            return 'UNKNOWN_PARSE_ERROR'    

        #revert morpheme list to ordered dict 
        m1 = self._get_dd(m1)
        m2 = self._get_dd(m2)
        
        #delete endings
        for v in ['глагольноеокончание', 'окончание', 'постфикс', 'нулевоеокончание', 'нулевое окончание', 'основа слова']:
            m1[v] = []
            m2[v] = []

        #get POS
        pos_pair = (self.get_POS(word1), self.get_POS(word2))
        
        #find dif
        dif = list(diff(m1, m2))
        if verbose: print(dif)
        
        #actual methods
        p = self._find(dif, action='add', morpheme='приставка') or self._find(dif, action='add', morpheme='префикс (приставка)') 
        s = self._find(dif, action='add', morpheme='суффикс') or self._find(dif, action='change', morpheme='суффикс') or self._find(dif, action='add', morpheme='интерфикс') or self._find(dif, action='change', morpheme='интерфикс')
        dk = self._find(dif, action='change', morpheme='корень')
        bs = self._find(dif, action='remove', morpheme='суффикс') or self._find(dif, action='remove', morpheme='интерфикс')
        
        ''' описание тегов из opencorpora
        NOUN	имя существительное	хомяк
        ADJF	имя прилагательное (полное)	хороший
        ADJS	имя прилагательное (краткое)	хорош
        COMP	компаратив	лучше, получше, выше ! WE COUNT IT AS ADJF !
        VERB	глагол (личная форма)	говорю, говорит, говорил
        INFN	глагол (инфинитив)	говорить, сказать
        PRTF	причастие (полное)	прочитавший, прочитанная
        PRTS	причастие (краткое)	прочитана
        GRND	деепричастие	прочитав, рассказывая
        NUMR	числительное	три, пятьдесят
        ADVB	наречие	круто
        NPRO	местоимение-существительное	он
        PRED	предикатив	некогда
        PREP	предлог	в
        CONJ	союз	и
        PRCL	частица	бы, же, лишь
        INTJ	междометие	ой
        
        '''
        # приставочный
        p_set = {('VERB', 'VERB'), ('VERB', 'INFN'), ('INFN', 'VERB'), ('INFN', 'INFN'),
                 ('NOUN', 'NOUN'), 
                 ('ADJF', 'ADJF'), ('ADJF', 'ADJS'), ('ADJS', 'ADJF'), ('ADJS', 'ADJS'),
                 ('NPRO', 'NPRO'),
                 ('ADVB', 'ADVB')}
        # суффиксальный
        s_set = {('NOUN', 'NOUN'), ('NOUN', 'ADJS'), ('NOUN', 'ADJF'), ('NOUN', 'VERB'), ('NOUN', 'INFN'),
                 ('VERB', 'VERB'), ('VERB', 'INFN'), ('VERB', 'NOUN'), ('VERB', 'ADJS'), ('VERB', 'ADJF'),
                 ('INFN', 'VERB'), ('INFN', 'INFN'), ('INFN', 'NOUN'), ('INFN', 'ADJS'), ('INFN', 'ADJF'),
                 ('ADJF', 'ADVB'), ('ADJF', 'NOUN'), ('ADJF', 'VERB'), ('ADJF', 'INFN'), ('ADJF', 'ADJF'),
                 ('ADJS', 'ADVB'), ('ADJS', 'NOUN'), ('ADJS', 'VERB'), ('ADJS', 'INFN'), ('ADJS', 'ADJS')}
        # приставочно-суффиксальный       
        ps_set = {('NOUN', 'NOUN'), 
                  ('NOUN', 'ADJF'), ('NOUN', 'ADJS'), 
                  ('NOUN', 'VERB'), ('NOUN', 'INFN'), ('ADJF', 'VERB'), ('ADJF', 'INFN'), ('ADJS', 'VERB'), ('ADJS', 'INFN'),
                  ('ADJF', 'ADVB'), ('ADJS', 'ADVB'), ('NOUN', 'ADVB'), ('NUMR', 'ADVB'),
                  ('INFN', 'INFN'), ('VERB', 'VERB'),# добавил я, надо чекать верно ли это
                  ('ADJF', 'CONJ'), ('ADJS', 'CONJ'),# добавил я, надо чекать верно ли это (пятый в-пятых)
                  ('ADVB', 'ADVB'),# добавил я, надо чекать верно ли это (век навеки
                 }
        # бессуфиксный    
        bs_set = {('INFN', 'NOUN'), ('VERB', 'NOUN'), 
                 ('ADJF', 'NOUN'), ('ADJS', 'NOUN'),
                 ('NOUN', 'NOUN'), 
                 ('NOUN', 'ADJF'), ('NOUN', 'ADJS'),
                 ('INFN', 'ADJF'), ('INFN', 'ADJS'), ('VERB', 'ADJF'), ('VERB', 'ADJS'),
                 ('ADJF', 'ADJF'), ('ADJF', 'ADJS'), ('ADJS', 'ADJF'), ('ADJS', 'ADJS'),
                 ('ADVB', 'ADJF'), ('ADVB', 'ADJS'),
                 ('NUMR', 'NUMR')}# проверить количественное-качественное
                 
        ## все далльше при условии что у нас только один корень!!!
        ## два надо отдельно смотреть
        if dk:
            #return "DIFF_ROOT"
            #теперь проверим вдруг жто чередующаяся согласыная
            if len(m1['корень'][0])==len(m2['корень'][0]):
                ss1 = {m1['корень'][0][-1], m2['корень'][0][-1]}
                ss2 = {m1['корень'][0][-1], m2['корень'][0][-2:]}
                ss3 = {m1['корень'][0][-2:], m2['корень'][0][-1]}
                ss4 = {m1['корень'][0][-2:], m2['корень'][0][-2:]}
                
                change_list = [{'г', 'ж'},{'г', 'жь'}, {'к', 'ч'}, {'д', 'ж'}, {'з', 'ж'}, {'к', 'ь'}, {'с', 'ш'},
                               {'т', 'ч'}, {'ч', 'т'}, {'ир', 'ор'}, {'ор', 'ар'}, {'ш', 'х'}, {'ит', 'ет'}]
                if (ss1 not in change_list) and (ss2 not in change_list) and (ss3 not in change_list) and (ss4 not in change_list):
                    return "DIFF_ROOT"   
        if (pos_pair in ps_set.union({pos_pair})) and p and s:
            if verbose: print(word1, '->', word2, ': приставочно-суффиксальный')
            return 'PS'
        if (pos_pair in p_set.union({pos_pair})) and  p:
            if verbose: print(word1, '->', word2, ': приставочный')
            return 'P'
        if (pos_pair in bs_set.union({pos_pair})) and (bs):
            if verbose: print(word1, '->', word2, ': безсуффиксный')
            return 'B'
        if (pos_pair in s_set.union({pos_pair})) and s:
            if verbose: print(word1, '->', word2, ': суффиксальный')
            return 'S'
        if (pos_pair in bs_set.union({pos_pair})) and (dk):
            if verbose: print(word1, '->', word2, ': безсуффиксный')
            return 'B'
        return 'UNKNOWN'

In [449]:
sp1 = SiteParser1()
sp3 = SiteParser3()

p = Analyzer([sp1, sp3])

In [363]:
TODO
посмотреть в обратном порядке - первое от второго или второе от первого
проверить части речи


SyntaxError: invalid syntax (<ipython-input-363-de77897935a2>, line 2)

In [420]:
p.analyze('лес', 'лесной', verbose=True)

[('add', '', [('суффикс', ['н'])])]
лес -> лесной : суффиксальный


'S'

In [450]:
test = pd.read_table('test.tsv', encoding='cp1251')

In [451]:
test.shape

(472, 4)

In [452]:
#test.drop_duplicates(keep='first', inplace=True)
#test.sort_values(by='label', inplace=True)
#test.reset_index(drop=True, inplace=True)
#test.to_csv('test.tsv', sep='\t')

In [453]:
res = []
for i, x in test.iterrows():
    print(i, x['word1'], x['word2'], end = ' ')
    res.append(p.analyze(x['word1'], x['word2'], True))

0 надрезать надрез [('remove', '', [('суффикс', ['а', 'ть'])])]
надрезать -> надрез : безсуффиксный
1 водить вождь [('change', ['корень', 0], ('вод', 'вождь')), ('remove', '', [('суффикс', ['и'])])]
водить -> вождь : безсуффиксный
2 лед колоть ледокол cant parse лед колоть, error:
3 сталь варить сталевар cant parse сталь варить, error:
4 подорвать подрыв [('change', ['приставка', 0], ('подо', 'под')), ('change', ['корень', 0], ('рв', 'рыв')), ('remove', '', [('суффикс', ['а'])])]
подорвать -> подрыв : безсуффиксный
5 отводить отвод [('remove', 'приставка', [(1, 'во')]), ('change', ['корень', 0], ('дить', 'вод'))]
отводить -> отвод : безсуффиксный
6 дарить дар [('remove', '', [('суффикс', ['и'])])]
дарить -> дар : безсуффиксный
7 взрывать взрыв [('remove', '', [('суффикс', ['а', 'ть'])])]
взрывать -> взрыв : безсуффиксный
8 гладкий гладь [('change', ['корень', 0], ('гладк', 'гладь'))]
гладкий -> гладь : безсуффиксный
9 глубокий глубь [('change', ['корень', 0], ('глубок', 'глубь'))]
глуб

85 загонять загон [('remove', '', [('суффикс', ['я'])])]
загонять -> загон : безсуффиксный
86 уличить улика [('change', ['корень', 0], ('улич', 'улик')), ('remove', '', [('суффикс', ['и'])])]
уличить -> улика : безсуффиксный
87 примешать примесь [('change', ['корень', 0], ('меш', 'месь')), ('remove', '', [('суффикс', ['а'])])]
примешать -> примесь : безсуффиксный
88 зажимать зажим [('remove', '', [('суффикс', ['а'])])]
зажимать -> зажим : безсуффиксный
89 заказать заказ [('remove', '', [('суффикс', ['а', 'ть'])])]
заказать -> заказ : безсуффиксный
90 закатиться заказ [('change', ['корень', 0], ('кат', 'заказ')), ('remove', '', [('приставка', ['за']), ('суффикс', ['и'])])]
закатиться -> заказ : безсуффиксный
91 залежаться залежь [('change', ['корень', 0], ('леж', 'лежь')), ('remove', '', [('суффикс', ['а'])])]
залежаться -> залежь : безсуффиксный
92 заливать залив cant parse заливать, error:
93 заложить залог cant parse залог, error:
94 нажимать нажим [('remove', '', [('суффикс', ['а'])

160 уколоть укол [('remove', '', [('суффикс', ['о'])])]
уколоть -> укол : безсуффиксный
161 обрывать обрыв [('change', ['суффикс', 0], ('ва', 'в')), ('remove', 'суффикс', [(1, 'ть')])]
обрывать -> обрыв : безсуффиксный
162 укладывать уклад [('remove', '', [('суффикс', ['ыва'])])]
укладывать -> уклад : безсуффиксный
163 ударить удар [('remove', '', [('суффикс', ['и', 'ть'])])]
ударить -> удар : безсуффиксный
164 уговориться уговор [('remove', '', [('суффикс', ['и'])])]
уговориться -> уговор : безсуффиксный
165 угореть угар [('change', ['корень', 0], ('гор', 'гар')), ('remove', '', [('суффикс', ['е'])])]
угореть -> угар : безсуффиксный
166 убирать убор [('change', ['корень', 0], ('бир', 'бор')), ('remove', '', [('суффикс', ['а'])])]
убирать -> убор : безсуффиксный
167 снарядить снаряд [('change', ['корень', 0], ('наряд', 'снаряд')), ('remove', '', [('приставка', ['с']), ('суффикс', ['и'])])]
снарядить -> снаряд : безсуффиксный
168 смыслить смысл [('remove', '', [('суффикс', ['и'])])]
смы

239 носить возносить [('add', '', [('приставка', ['воз'])])]
носить -> возносить : приставочный
240 модный сверхмодный [('add', '', [('приставка', ['сверх'])])]
модный -> сверхмодный : приставочный
241 лево налево [('add', '', [('приставка', ['на']), ('суффикс', ['о'])])]
лево -> налево : приставочно-суффиксальный
242 когда никогда [('add', '', [('приставка', ['ни'])])]
когда -> никогда : приставочный
243 кто никто [('add', '', [('приставка', ['ни'])])]
кто -> никто : приставочный
244 бежать сбежать [('add', '', [('приставка', ['с'])])]
бежать -> сбежать : приставочный
245 решать разрешать [('change', ['корень', 0], ('реш', 'разреш'))]
решать -> разрешать : безсуффиксный
246 делать сделать [('add', '', [('приставка', ['с'])])]
делать -> сделать : приставочный
247 плескать выплескать [('add', '', [('приставка', ['вы'])])]
плескать -> выплескать : приставочный
248 грызть загрызть [('add', '', [('приставка', ['за'])])]
грызть -> загрызть : приставочный
249 бежать прибежать [('add', '', [(

320 звук беззвучный [('change', ['корень', 0], ('звук', 'звуч')), ('add', '', [('приставка', ['без']), ('суффикс', ['н'])])]
звук -> беззвучный : приставочно-суффиксальный
321 палец напальчник cant parse напальчник, error:
322 береза подберезовик cant parse береза, error:
323 облако заоблачный [('change', ['корень', 0], ('облак', 'облач')), ('add', '', [('приставка', ['за']), ('суффикс', ['н'])])]
облако -> заоблачный : приставочно-суффиксальный
324 кровать прикроватный [('change', ['корень', 0], ('кровать', 'кроват')), ('add', '', [('приставка', ['при']), ('суффикс', ['н'])])]
кровать -> прикроватный : приставочно-суффиксальный
325 школа пришкольный [('change', ['корень', 0], ('школ', 'школь')), ('add', '', [('приставка', ['при']), ('суффикс', ['н'])])]
школа -> пришкольный : приставочно-суффиксальный
326 рука наручники [('change', ['корень', 0], ('рук', 'руч')), ('add', '', [('приставка', ['на']), ('суффикс', ['ник'])])]
рука -> наручники : приставочно-суффиксальный
327 перый во-перв

389 парашют парашютист [('add', '', [('суффикс', ['ист'])])]
парашют -> парашютист : суффиксальный
390 учить учитель [('add', 'суффикс', [(1, 'тель')])]
учить -> учитель : суффиксальный
391 храбрый храбрость [('add', '', [('суффикс', ['ость'])])]
храбрый -> храбрость : суффиксальный
392 строить строение cant parse строить, error:
393 стол столик [('add', '', [('суффикс', ['ик'])])]
стол -> столик : суффиксальный
394 забота заботливый [('add', '', [('суффикс', ['лив'])])]
забота -> заботливый : суффиксальный
395 весна весенний [('change', ['корень', 0], ('весн', 'весен')), ('add', '', [('суффикс', ['н'])])]
весна -> весенний : суффиксальный
396 серый сероватый [('add', '', [('суффикс', ['оват'])])]
серый -> сероватый : суффиксальный
397 гусь гусиный [('change', ['корень', 0], ('гусь', 'гус')), ('add', '', [('суффикс', ['ин'])])]
гусь -> гусиный : суффиксальный
398 медленный медленно [('add', 'суффикс', [(1, 'о')])]
медленный -> медленно : суффиксальный
399 бесшумный бесшумно cant parse 

In [454]:
len(res), test.shape

(472, (472, 4))

In [455]:
test['res'] = res

In [456]:
p.get_POS('уплатить'), p.get_POS('уплата')

('INFN', 'NOUN')

In [457]:
test.head(10)

,Unnamed: 0,word1,word2,label,res
0,0,надрезать,надрез,B,B
1,1,водить,вождь,B,B
2,2,лед колоть,ледокол,B,UNKNOWN_PARSE_ERROR
3,3,сталь варить,сталевар,B,UNKNOWN_PARSE_ERROR
4,4,подорвать,подрыв,B,B
5,5,отводить,отвод,B,B
6,6,дарить,дар,B,B
7,7,взрывать,взрыв,B,B
8,8,гладкий,гладь,B,B
9,9,глубокий,глубь,B,B


In [458]:
test.label.value_counts()

B     230
S      98
P      82
PS     58
PF      4
Name: label, dtype: int64

In [459]:
print('accuracy measuring')

total_acc = sum(test.label==test.res) / test.shape[0]
print('total accuracy: ', total_acc)

for l in test.label.unique():
    filt = test.label==l
    tmp_acc = sum(test[filt].label==test[filt].res) / (test[filt].shape[0] +0.1)
    print(l, ' accuracy: ', tmp_acc)

accuracy measuring
total accuracy:  0.792372881356
B  accuracy:  0.83876575402
P  accuracy:  0.755176613886
PF  accuracy:  0.0
PS  accuracy:  0.705679862306
S  accuracy:  0.795107033639


In [460]:
test[test.res=='UNKNOWN_PARSE_ERROR'].shape

(52, 5)

In [447]:
p.analyze('река', 'речной')

[('change', ['корень', 0], ('рек', 'реч')), ('add', '', [('суффикс', ['н'])])]
река -> речной : безсуффиксный


'B'

In [445]:
sp1.get_morphems('страдать'), sp3.get_morphems('страдать'), sp1.get_morphems('сострадать'), sp3.get_morphems('сострадать'), 

([('страда', 'корень'), ('ть', 'глагольноеокончание')],
 [('страда', 'корень'), ('ть', 'суффикс')],
 [('со', 'приставка'),
  ('страд', 'корень'),
  ('а', 'суффикс'),
  ('ть', 'глагольноеокончание')],
 [('со', 'приставка'), ('страда', 'корень'), ('ть', 'постфикс')])

In [443]:
p.get_POS('сколько'), p.get_POS('несколько'), 

('CONJ', 'ADVB')

In [ ]:
лис	лиса	
кум	кума
золото	золотой

In [ ]:
['кое-', 'ультра', 'контр']

In [461]:
test[(test.label!=test.res) & (test.res!='UNKNOWN_PARSE_ERROR')]

,Unnamed: 0,word1,word2,label,res
13,13,кум,кума,B,UNKNOWN
14,14,лис,лиса,B,UNKNOWN
17,17,золото,золотой,B,UNKNOWN
21,21,ниже,низший,B,S
33,33,вызывать,вызов,B,DIFF_ROOT
47,47,подвод,подводить,B,S
48,48,разъезд,разъезжать,B,S
56,56,высевать,высев,B,S
121,121,прясть,пряжа,B,S
123,123,пересматривать,пересмотр,B,DIFF_ROOT


In [250]:
p.analyze('бежать', 'бежал')

[('remove', '', [('корень', ['беж']), ('суффикс', ['а'])])]


'UNKNOWN'

In [252]:
sp1.get_morphems('бежать'), sp1.get_morphems('бежал'), 

([('беж', 'корень'), ('а', 'суффикс'), ('ть', 'глагольноеокончание')],
 {'error': 'site parsing error'})

In [254]:
sp3.get_morphems('бежать'), sp3.get_morphems('бежал'), 

([('беж', 'корень'), ('а', 'суффикс'), ('ть', 'суффикс')], [])

In [221]:
p._find([('change', ['корень', 0], ('беж', 'бег')), ('remove', '', [('суффикс', ['а'])])], 'add', 'суффикс')

False

In [234]:
dif = [('add', '', [('приставка', ['в']), ('суффикс', ['ых'])])]

In [237]:
s = p._find(dif, action='add', morpheme='суффикс')

In [238]:
s

True